<a href="https://colab.research.google.com/github/colabllmdi/travel_bot/blob/main/TravelBot_RAG_ChromaDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers arxiv auto_gptq tiktoken chromadb langchain accelerate chardet cchardet torch einops accelerate SentencePiece sentence_transformers pinecone-client datasets preprocess gradio xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 84.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 83.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.9/421.9 kB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 87.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 653.6/653.6 kB 57.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.1/179.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from transformers import LlamaForCausalLM, LlamaTokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
import langchain
import os
import gradio as gr
import pandas as pd
from torch import cuda
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
import pinecone
import time
from langchain.chains import ConversationalRetrievalChain, ConversationChain
from langchain.memory import ConversationBufferWindowMemory
from langchain.llms import HuggingFacePipeline
from langchain.vectorstores import Pinecone

# Imports
from chromadb.config import Settings
from urllib.error import HTTPError
from dataclasses import replace
from dotenv import load_dotenv
from tqdm import tqdm
import pandas as pd
import chromadb
from chromadb.utils import embedding_functions
import numpy as np
import tiktoken # OpenAI's open-source tokenizer
import chromadb
import logging
import random # to sample multiple elements from a list
import arxiv
import time
import os # operating system dependent functionality, to walk through directories and files
from transformers import LlamaForCausalLM, LlamaTokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
import langchain
import gradio as gr
import pandas as pd
from torch import cuda
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
import time
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory
from langchain.vectorstores import Pinecone
from langchain.text_splitter import RecursiveCharacterTextSplitter # recursively tries to split by different characters to find one that works
from langchain.document_loaders import PyPDFDirectoryLoader # loads pdfs from a given directory
from langchain.chains import ConversationalRetrievalChain # looks up relevant documents from the retriever per history and question.
from langchain.text_splitter import CharacterTextSplitter # splits the content
from langchain.embeddings import HuggingFaceBgeEmbeddings # wrapper for HuggingFaceBgeEmbeddings models
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from langchain.document_loaders import ArxivLoader # loads paper for a given id from Arxiv
from langchain.document_loaders import PyPDFLoader # loads a given pdf
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader # loads a given text
from langchain.retrievers import ArxivRetriever # loads relevant papers for a given paper id from Arxiv
from chromadb.utils import embedding_functions # loads Chroma's embedding functions from OpenAI, HuggingFace, SentenceTransformer and others
from langchain.chat_models import ChatOpenAI # wrapper around OpenAI LLMs
from langchain.vectorstores import Chroma # wrapper around ChromaDB embeddings platform
from langchain.chains import RetrievalQA
from langchain.chains import RetrievalQAWithSourcesChain
from langchain import HuggingFaceHub # wrapper around HuggingFaceHub models

from transformers import AutoTokenizer, pipeline, logging
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Sep 11 10:17:33 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    45W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    n_gpu = torch.cuda.device_count()
    torch.cuda.get_device_name(0)
else:
    device = torch.device("cpu")

In [ ]:
hotel = pd.read_csv(r'/content/drive/MyDrive/MoT Data Sample/Riyadh_Hotel_Sample.csv')
restaurant = pd.read_csv(r'/content/drive/MyDrive/MoT Data Sample/Riyadh_Restaurant_Sample.csv')
mall = pd.read_csv(r'/content/drive/MyDrive/MoT Data Sample/Riyadh_Mall_Sample.csv')
tourist = pd.read_csv(r'/content/drive/MyDrive/MoT Data Sample/Riyadh_Tourist_Sample.csv')

In [ ]:
hotel['types'] = 'Hotel'
restaurant['types'] = 'Restaurant'
mall['types'] = 'Shopping Mall'
tourist['types'] = 'Place of Tourist Attraction'

In [ ]:
def create_statement(df):
  df['Statement_Description'] = df['name'] + ' is a ' + df['types']
  df['Statement_Address'] = "Address for " + df['types'] + " " + df['name'] + " is " + df['formatted_address']
  df['Statement_Rating'] = "Rating for " + df['types'] + " " + df['name'] + " is " + df['rating'].astype(str) + " out of " + df['user_ratings_total'].astype(str) + " user ratings"
  return(df)

hotel = create_statement(hotel)
restaurant = create_statement(restaurant)
mall = create_statement(mall)
tourist = create_statement(tourist)

data_converted = pd.DataFrame()
data_converted = pd.concat([hotel, restaurant, mall, tourist]).reset_index()
data_converted = data_converted.drop(columns=['index', 'name', 'formatted_address', 'rating', 'user_ratings_total', 'types',
       'place_id', 'price_level'])

Statement_Description = data_converted['Statement_Description'].tolist()
Statement_Address = data_converted['Statement_Address'].tolist()
Statement_Rating = data_converted['Statement_Rating'].tolist()

statements = Statement_Description + Statement_Address + Statement_Rating

cleanedList = [x for x in statements if str(x) != 'nan']

In [ ]:
import chromadb
chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name="Riyadh_Travel_Data", metadata={"hnsw:space": "cosine"})

In [ ]:
model_id = "/content/drive/MyDrive/all-MiniLM-L6-v2"

In [45]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer(model_id)

In [46]:
embeddings = model.encode(statements)

In [ ]:
id= [str(x) for x in list(range(0,len(cleanedList)))]

In [ ]:
for i in range(len(cleanedList)):
  collection.add(
    embeddings=[embeddings[i].tolist()],
    documents=cleanedList[i],
    ids=id[i]
)

In [ ]:
model_id_llama_2 = "/content/drive/MyDrive/LLM_Models/Llama-2-7b-chat-hf/"
tokenizer = AutoTokenizer.from_pretrained(model_id_llama_2, use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:640: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [ ]:
llama_2_model = AutoModelForCausalLM.from_pretrained(model_id_llama_2).to(device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [ ]:
from transformers import pipeline

llama_pipeline = pipeline(
    "text-generation",
    model=model_id_llama_2,
    torch_dtype=torch.float16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [26]:
###### User Prompt Analysis


SYSTEM_PROMPT = '''
          You are a classification agent.
          You are supposed to identify the user's requirement based on the prompt input.
          You can classify the user prompt into three categories of Description, Itinerary and Others only.
          Following are the examples you can utilize:

           User: Recommend some Arabic phrases I should know when visiting Riyadh
           Assistant: Others

           User: Can you suggest a reliable hotel booking website for Riyadh?
           Assistant: Others

           User: What's the currency exchange rate in Riyadh?
           Assistant: Others

           User: Are there any upcoming events or festivals in Riyadh?
           Assistant: Others

           User: Tell me about the local transportation options within Riyadh
           Assistant: Others

           User: What's the best way to travel from Riyadh to Jeddah?
           Assistant: Others

           User: How do I obtain a tourist visa for Saudi Arabia?
           Assistant: Others

           User: Create a itineray for 2 days in Riyadh, include good rated restaurants near the hotel stay and places to explore around the hotel. Also, find me good malls for shopping near the hotel
           Assistant: Itinerary

           User: I want to go to Riyadh for a trip, can you help me?
           Assistant: Itinerary

           User: Plan me a trip to Riyadh for 5 days
           Assistant: Itinerary

           User: What should I do for 2 days in Riyadh
           Assistant: Itinerary

           User: Create an itinerary for 3 days in Riyadh, choose the area with good restaurant ratings
           Assistant: Itinerary

           User: Is Al Masmak fort worth visiting?
           Assistant: Description

           User: What is Riyadh Season?
           Assistant: Description

           User: How is View Mall better than Riyadh Park mall?
           Assistant: Description

           User: Is Al Nakheel Mall near crowne plaza hotel ?
           Assistant: Description

           User: Is Golden Dragon a good restaurant
           Assistant: Description

           User: How is Piatto Restaurant for lunch?
           Assistant: Description

           User: Create an itinerary for 4 days in Riyadh near Crowne Plaza Hotel
           Assistant: Itinerary

           User: Create an itinerary for 2 days in Riyadh
           Assistant: Itinerary

           User: How is Narcissus Hotel for me?
           Assistant: Description

    '''

def generate_prompt(prompt: str, system_prompt: str = SYSTEM_PROMPT) -> str:
    return f"""
      [INST] <<SYS>>
      {system_prompt}
      <</SYS>>

      {prompt} [/INST]
      """.strip()

In [28]:
from langchain import HuggingFacePipeline, PromptTemplate

template = generate_prompt(
    """
{context}

Question: {question}
""",
    system_prompt=SYSTEM_PROMPT,
)

prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [29]:
generate_text = transformers.pipeline(
    model=llama_2_model, tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    temperature=0.0,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=512,  # max number of tokens to generate in the output
    repetition_penalty=1.1,
    device=device# without this output begins repeating
)

In [30]:
from langchain.chains import RetrievalQA
llm = HuggingFacePipeline(pipeline=generate_text)

AttributeError: ignored

AttributeError: ignored

In [31]:
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)
# retrieval qa chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=collection.as_retriever(),
    memory = conversational_memory,
    chain_type_kwargs={"prompt": prompt}
)
q_p = ConversationalRetrievalChain.from_llm(
        llm=llm,
        chain_type='stuff',
        retriever=collection.as_retriever(),
        memory=conversational_memory,
        get_chat_history=lambda h: h,
        verbose=True
    )

AttributeError: ignored

In [ ]:
results = collection.query(
    query_texts=[prompt],
    n_results=3,
    include=["embeddings", "documents", "metadatas", "distances"],
)
# results

In [ ]:
ask = "Itinerary"

if ask == 'Description':
  SYSTEM_PROMPT = '''
    You are a travel assistant. Which describes a location as per the user ask.
    Your job is to answer the user prompt as a description as per the given context {}.
    Description should be consise and in bullets.
    Do not use any other data outside of the following context: {}
    '''
elif ask == 'Itinerary':
  SYSTEM_PROMPT = '''

    You are a travel assistant which creates travel itineraries.
    Assistant will create itinerary which will include a hotel stay for the entire trip duration and visits to minimum two restaurants and one tourist attraction daily.
    Assistant will optimise the travel itinerary in a way that hotel stay is for the entire trip duration and rest of the info is according to the location of hotel.
    Assistant will provide the output in a tabular format with one row for each day of the itinerary.
    '''
else:
  SYSTEM_PROMPT = '''

    Provide output as: "Sorry, I don't have the necessary data for that. Kindly ask again!"
    '''



In [ ]:
def generate_prompt(prompt: str, system_prompt: str = SYSTEM_PROMPT) -> str:
    return f"""
[INST] <<SYS>>
{system_prompt}
<</SYS>>

{prompt} [/INST]
""".strip()